# Open AIモデルのファインチューニング

このノートブックは、Open AIの[ファインチューニング](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst)ドキュメントに基づいています。

ファインチューニングは、特定のユースケースやシナリオに関連する追加データやコンテキストでモデルを再学習させることで、基盤モデルのパフォーマンスをアプリケーション向けに向上させる手法です。_few shot learning_ や _retrieval augmented generation_ のようなプロンプトエンジニアリングのテクニックを使えば、関連データをプロンプトに追加して品質を高めることができます。ただし、これらの方法は対象となる基盤モデルの最大トークンウィンドウサイズに制限されます。

ファインチューニングでは、必要なデータでモデル自体を再学習させるため（最大トークンウィンドウに収まるよりも多くの例を使うことが可能）、推論時に例を毎回与える必要のない_カスタム_バージョンのモデルをデプロイできます。これにより、プロンプト設計の柔軟性が高まるだけでなく（トークンウィンドウを他の用途に使える）、推論時にモデルへ送るトークン数を減らすことでコスト削減にもつながる可能性があります。

ファインチューニングは4つのステップで行います：
1. トレーニングデータを準備してアップロードする
1. トレーニングジョブを実行してファインチューニング済みモデルを作成する
1. ファインチューニング済みモデルを評価し、品質向上のために繰り返す
1. 満足できる品質になったら、推論用にファインチューニング済みモデルをデプロイする

すべての基盤モデルがファインチューニングに対応しているわけではありません。最新情報は[OpenAIのドキュメント](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst)を確認してください。また、以前にファインチューニングしたモデルをさらにファインチューニングすることも可能です。このチュートリアルでは、ファインチューニングの対象となる基盤モデルとして `gpt-35-turbo` を使用します。

---


### ステップ 1.1: データセットの準備

周期表の元素について質問に答える際、リメリックで答えてくれるチャットボットを作ってみましょう。このシンプルなチュートリアルでは、モデルをトレーニングするためのデータセットを、期待されるデータ形式を示すいくつかのサンプル応答例で作成します。実際のユースケースでは、もっと多くの例を含むデータセットが必要になります。もし該当するオープンデータセットがあれば、それを使ってフォーマットを調整し、ファインチューニング用に利用することもできます。

今回は `gpt-35-turbo` を対象に、1回のやりとり（チャット完了）で返答する形式を想定しているので、[この推奨フォーマット](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst)を使って、OpenAIのチャット完了要件に沿った例を作成できます。もし複数ターンの会話を想定する場合は、[マルチターン用の例のフォーマット](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst)を使い、`weight` パラメータでファインチューニング時にどのメッセージを使うか（使わないか）を指定します。

このチュートリアルでは、よりシンプルな1ターン形式を使います。データは [jsonl形式](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) で、1行に1レコード、各レコードはJSON形式のオブジェクトで表現されます。下記のスニペットは2つのレコードのサンプルです。完全なサンプルセット（10例）は [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) で確認できます。**注意:** 各レコードは必ず1行で定義してください（通常のJSONファイルのように複数行に分割しないでください）

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

実際のユースケースでは、良い結果を得るためにもっと多くの例が必要です。応答の質とファインチューニングにかかる時間・コストのバランスを考える必要があります。ここでは、プロセスを説明するために少数の例で素早くファインチューニングできるようにしています。より複雑なファインチューニングのチュートリアルについては、[OpenAI Cookbookの例](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) を参照してください。


### ステップ 1.2 データセットのアップロード

Files API を使ってデータをアップロードします（[こちらの説明](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file)を参照してください）。このコードを実行する前に、以下の手順を完了している必要があります：
 - `openai` Python パッケージをインストールしていること（最新機能を使うにはバージョン >=0.28.0 を使用してください）
 - `OPENAI_API_KEY` 環境変数に OpenAI API キーを設定していること
詳しくは、コース用に用意された[セットアップガイド](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst)をご覧ください。

次に、ローカルの JSONL ファイルからアップロード用のファイルを作成するコードを実行します。


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### ステップ 2.1: SDKでファインチューニングジョブを作成する


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### ステップ 2.2: ジョブのステータスを確認する

`client.fine_tuning.jobs` API でできることは以下の通りです：
- `client.fine_tuning.jobs.list(limit=<n>)` - 直近の n 件のファインチューニングジョブを一覧表示
- `client.fine_tuning.jobs.retrieve(<job_id>)` - 特定のファインチューニングジョブの詳細を取得
- `client.fine_tuning.jobs.cancel(<job_id>)` - ファインチューニングジョブをキャンセル
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - ジョブの最新 n 件のイベントを一覧表示
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

このプロセスの最初のステップは、_トレーニングファイルの検証_ です。データが正しい形式になっているかを確認します。


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### ステップ 2.3: 進捗を把握するためにイベントをトラッキングする


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### ステップ2.4：OpenAIダッシュボードでステータスを確認する


OpenAIのウェブサイトにアクセスし、プラットフォームの _Fine-tuning_ セクションを確認することで、ステータスを確認することもできます。ここでは現在のジョブの状況が表示され、過去のジョブ実行履歴も追跡できます。このスクリーンショットでは、前回の実行が失敗し、2回目の実行が成功したことが分かります。参考までに説明すると、最初の実行ではJSONファイルのレコード形式に誤りがあったため失敗しましたが、修正後の2回目の実行は正常に完了し、モデルが利用可能になりました。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.ja.png)


さらに下にスクロールすると、ビジュアルダッシュボードでステータスメッセージやメトリクスも確認できます。

| メッセージ | メトリクス |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.ja.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.ja.png)|


### ステップ 3.1: IDの取得とファインチューニング済みモデルのコードでのテスト


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### ステップ 3.2: Playground でファインチューニング済みモデルを読み込み・テストする

ファインチューニング済みモデルは、2つの方法でテストできます。まず、Playground にアクセスし、モデルのドロップダウンから新しくファインチューニングしたモデルを選択できます。もう一つの方法は、ファインチューニングパネルに表示されている「Playground」オプションを使うことです（上記のスクリーンショット参照）。これを使うと、基盤モデルとファインチューニング済みモデルを並べて比較できるビューが起動し、素早く評価できます。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.ja.png)

トレーニングデータで使ったシステムコンテキストを入力し、テスト用の質問を入力してください。両方のモデルに同じコンテキストと質問が反映されるのが分かります。比較を実行すると、それぞれの出力の違いが確認できます。_ファインチューニング済みモデルは、例で指定したフォーマットで回答を返すのに対し、基盤モデルはシステムプロンプトに従うだけである点に注目してください_。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.ja.png)

また、比較画面では各モデルのトークン数や推論にかかった時間も表示されます。**この例はプロセスを説明するためのシンプルなものなので、実際のデータセットや現実のシナリオを反映しているわけではありません**。両方のサンプルでトークン数が同じ（システムコンテキストとユーザープロンプトが同一）ですが、ファインチューニング済みモデル（カスタムモデル）の方が推論に時間がかかっているのが分かるかもしれません。

実際の現場では、このような簡単な例ではなく、実データ（例：カスタマーサービス用の商品カタログなど）を使ってファインチューニングを行うため、回答の質の違いがより明確になります。その場合、基盤モデルで同等の回答品質を得るには、より高度なプロンプトエンジニアリングが必要となり、トークン消費量や推論にかかる処理時間が増える可能性があります。_実際に試してみたい場合は、OpenAI Cookbook のファインチューニング例を参考にしてください。_



---

**免責事項**:  
本書類は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性には努めておりますが、自動翻訳には誤りや不正確な表現が含まれる場合があります。原文（元の言語の文書）が正式な情報源と見なされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じたいかなる誤解や誤訳についても、当方は一切の責任を負いかねます。
